<a href="https://colab.research.google.com/github/dinesh110598/IsingMCMC-tensorflow/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ising MCMC with Tensorflow**

Here we construct a graph in Tesnorflow 2.0 that performs a MCMC simulation of Ising model at different temperatures and plot the behaviour of magnetisation with temperature

Let's first import the required libraries!


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
import os
import itertools
%matplotlib inline

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow.math as tfm
import tensorflow as tf

Now, we shall define a class that will manipulate spin lattices. It shall include a method that calculates lattice energy according to the Ferromagnetic Ising Hamiltonian:
\begin{align}
H = -J \displaystyle\sum_{bonds}(S^z_i S^z_{i+1}) 
\end{align}
We will use a more speed-optimized Monte Carlo Ising model algorithm as compared to the rudimentary ones used in previous versions. Also, we look at an ensemble of size 4 x 4 of Ising lattices of size 256 X 256 each rather than individual lattices alone, in the aim of being more compatible with concepts in statistical physics, like order parameters being ensemble averages rather than over individual lattices. $ \sigma $

Let's first define some constants of the model:

In [ ]:
def create_iterator(shape):
    """Create an iterator with a given shape."""
    dims = [range(dim) for dim in shape]
    return itertools.product(*dims)

def create_list(shape):
    """Create a list with a given shape and default value None."""
    if shape:
        return [create_list(shape[1:]) for _ in range(shape[0])]

lattice_len = 128
SUBLAT_LEN = lattice_len // 2
#Length of sublattice above defines the size of square containing a particular
#color alone, say black or white, in the optimal version of checkerboard algm

kernel = np.eye(SUBLAT_LEN, k=0, dtype=np.float32) 
kernel += np.eye(SUBLAT_LEN, k=1, dtype=np.float32)
kernel [SUBLAT_LEN-1][0] = np.float32 (1)
#This helps us implement periodic boundary conditions on each lattice rather
#the entire ensemble as suggested in the original alogorithm

#End of constants

## GPU optimization:

This is the function that traces an autograph to optimize the resource-heavy calculations:

In [ ]:
with tf.device('/GPU:0'):
    ensemble = create_list (ens_shape[:2])
    for i, j in create_iterator ([2,2]):
        ensemble[i][j] = tf.Variable (tf.ones (ens_shape[2:], tf.float32))
    KERNEL_MAT = tf.constant (kernel, tf.float32)
    ens_shape = [2,2,40,SUBLAT_LEN,SUBLAT_LEN]

@tf.function
def equilib (iter, T):
    def get_inverse(T):
        """Returns the inverse of a number."""
        return tfm.reciprocal (T)

    def sum_of_nearest_neighbors (black):
        if black == True:
            sum_nn_00 = (tf.matmul(ensemble[0][1], KERNEL_MAT) +
            tf.matmul(KERNEL_MAT, ensemble[1][0], transpose_a=True))
            sum_nn_11 = (tf.matmul(KERNEL_MAT, ensemble[0][1]) +
            tf.matmul(ensemble[1][0], KERNEL_MAT, transpose_b=True))
            return sum_nn_00, sum_nn_11
        else:
            sum_nn_01 = (tf.matmul(ensemble[0][0], KERNEL_MAT, transpose_b=True) +
                    tf.matmul(KERNEL_MAT, ensemble[1][1], transpose_a=True))
            sum_nn_10 = (tf.matmul(KERNEL_MAT, ensemble[0][0]) +
                        tf.matmul(ensemble[1][1], KERNEL_MAT))
            return sum_nn_01, sum_nn_10

    def update_optim (T):
        def update (probs, black):
            if black:
                idx = [[0, 0], [1, 1]]
            else:
                idx = [[0, 1], [1, 0]]

            sum_nn_color = sum_of_nearest_neighbors (black)

            for [idx0, idx1], sum_nn in zip(idx, sum_nn_color):
                acceptance_ratio = (-2 * get_inverse (T) * sum_nn *
                                    ensemble[idx0][idx1])
                flips = tf.cast(probs[idx0][idx1] < acceptance_ratio,
                                dtype= tf.float32)
            #Instead of demanding probabilities be less than exp(-2*sum_nn/T) for 
            #all spins, we take an equivalent condition of log (prob) < -2*sum_nn/T
                ensemble[idx0][idx1].assign_sub (flips*ensemble[idx0][idx1]*2)

        probs = tf.math.log(tf.random.uniform(ens_shape, dtype=tf.float32))
        update(probs, black=True)
        update(probs, black=False)
        return ensemble
    
    print ("Traced")

    #Initialize the Ising lattice to +1s
    for i, j in create_iterator ([2,2]):
        ensemble[i][j].assign (tf.ones (ens_shape[2:], tf.float32))

    for _ in tf.range(iter):
        update_optim (T)
    return ensemble

Now, we are ready to calculate the average equilibrium magnetisations at different temperatures, with stepsize $ 0.1 k_B $. The data for magnetization and corresponding temperatures are collected using two lists:

In [ ]:
#Contains list of variables describing
#lattices for all temperature data points

def generate_graph (upper_t = 3., lower_t = 1.5, step = 0.1):
    temp = lower_t
    _len_ = int((upper_t - lower_t) // step)
    data = np.empty((2,_len_), dtype=np.float32)
    iter = tf.constant (300)
    val = np.empty ((2,2),np.float32)
    for n in range (_len_):
        ensemble = equilib (iter, tf.constant(temp))
        for i,j in create_iterator ([2,2]):
            val[i][j] = ensemble[i][j].numpy().mean()
        data[0][n] = temp
        data[1][n] = val.mean()
        temp += step
        #print ("Iteration", n)
    #graph, ax = plt.subplots()
    #ax.plot (data[0], data[1], 'r-', linewidth = 2, label = 'Magnetisation')
    #ax.legend (loc = 'upper right')
    #ax.grid()
    #plt.show()

y = equilib (300, tf.constant (0.8))

Traced


In [ ]:
x1 = tf.ones (ens_shape, tf.float32)
x2 = tf.random.uniform (ens_shape[2:])
print (tf.matmul (x1[1,0], x2).shape)

(40, 64, 64)


Let's compare the speed with numba-cuda based simulation:

In [ ]:
from numba import cuda
from time import time
import functools
import math

def timeit(n=10):
    """
    Decorator to run function n times and   print out the total time elapsed.
    """
    def dec(func):
        @functools.wraps(func)
        def wrapped(*args, **kwargs):
            t0 = time()
            for i in range(n):
                func(*args, **kwargs)
            print("%s iterated %d times\nTime elapsed %.3fs\n" % (
                func.__name__, n, time() - t0))
        return wrapped
    return dec

In [ ]:
threadsperblock = (8,8,1)
blockspergrid = (16,16,40)
shape = (128,128)
ens_size = 40

@cuda.jit
def update_black (spin, seed, T):

    def random_uniform (x,y,z):
        seed[x,y,z] = np.int32((seed[x,y,z]*1664525 + 1013904223) % 2**31)
        return seed[x,y,z] / (2**31)

    def bvc (x):
        return 0 if x == spin.shape[0] else x
    
    def sum_nn (x, y, z):
        value = 0
        value += spin [bvc(x+1), y, z]
        value += spin [x-1, y, z]
        value += spin [x, bvc(y+1), z]
        value += spin[x, y-1, z]
        return value

    def calc (x, y, z):
        probs = random_uniform (x, y, z)
        if (probs < math.exp(-2*spin[x,y,z]*sum_nn(x,y,z)/T[0])):
            spin[x,y,z] *= np.int8(-1)
    
    x,y,z= cuda.grid(3)
    p,q = x%2,y%2

    if ((p==0 and q==0) or (p==1 and q==1)):
        calc (x,y,z)

@cuda.jit
def update_white (spin, seed, T):

    def random_uniform (x,y,z):
        seed[x,y,z] = np.int32((seed[x,y,z]*1664525 + 1013904223) % 2**31)
        return seed[x,y,z] / (2**31)

    def bvc (x):
        return 0 if x == spin.shape[0] else x
    
    def sum_nn (x, y, z):
        value = 0
        value += spin [bvc(x+1), y, z]
        value += spin [x-1, y, z]
        value += spin [x, bvc(y+1), z]
        value += spin[x, y-1, z]
        return value

    def calc (x, y, z):
        probs = random_uniform (x, y, z)
        if (probs < math.exp(-2*spin[x,y,z]*sum_nn(x,y,z)/T[0])):
            spin[x,y,z] *= np.int8(-1)
    
    x,y,z = cuda.grid(3)
    p,q = x%2,y%2

    if ((p==0 and q==1) or (p==1 and q==0)):
        calc (x,y,z)

def equilibrate (temp):
    spin = np.ones (shape+(ens_size,), dtype= np.int8)
    seed = np.random.randint (-10000, 10000, size=shape+(ens_size,),
                              dtype=np.int32)
    vr_spin = cuda.to_device (spin)
    vr_seed = cuda.to_device (seed)
    vr_temp = cuda.to_device (temp)
    for _ in range (300):
        update_black[blockspergrid,threadsperblock] (vr_spin, vr_seed, vr_temp)
        update_white[blockspergrid,threadsperblock] (vr_spin, vr_seed, vr_temp)
    spin = vr_spin.copy_to_host()
    return spin

def mag_data (low, high, step):
    _len = math.ceil ((high-low) / step)
    data_x = np.empty (_len, dtype=np.float32)
    data_y = np.empty (_len, dtype=np.float32)
    mags = np.empty (_len, dtype=np.float32)
    temp = np.float32(low)
    
    for i in range (_len): 
        data_x[i] = temp
        mags[i] = abs (np.mean (equilibrate(temp)))
        temp += step 
    #graph, ax = plt.subplots()
    #ax.plot (data_x, mags, 'r-', linewidth = 2)
    #ax.grid()
    #plt.show()

In [ ]:
def compute_cuda():
    mag_data (1.5,2.5,0.1)

def compute_tflow():
    generate_graph (2.5,1.5,0.1)

timeit()(compute_cuda)()
timeit()(compute_tflow)()

compute_cuda iterated 10 times
Time elapsed 21.565s

Tracing
compute_tflow iterated 10 times
Time elapsed 39.728s



Tensorflow graphs are slightly slower compared to Numba CUDA since explicit CUDA codes are **compiled** while graphs are aren't exactly so (the finer details are beyond my head). If there's any need to accelerate tensorflow graphs by compilation, [XLA](https://https://www.tensorflow.org/xla) is one such compiler natively supported by Tensorflow. Using XLA can potentially narrow down the speed difference between plain CUDA and TF graphs.